In [91]:
import pandas as pd
import numpy as np

In [92]:
def prepare(data):
    
    buildtimes = np.array([1.385, 0.696, 0.766, 0.689, 0.833])
    
    data['buldingtime'] = data['buldingtime'] + buildtimes.mean()
    
    data['import.start'] = pd.to_datetime(data['import.start'])
    data['import.stop'] = pd.to_datetime(data['import.stop'])
    data['created'] = pd.to_datetime(data['created'])
    data['started'] = pd.to_datetime(data['started'])
    data['finished'] = pd.to_datetime(data['finished'])
    
    data['starttime'] = (data['started'] - data['created']).dt.total_seconds() 
    data['dockertime'] = (data['finished'] - data['started']).dt.total_seconds() 
    data['importtime'] = (data['import.stop'] - data['import.start']).dt.total_seconds() 
    
    started_temp = pd.DataFrame()
    docker_temp = pd.DataFrame()
    import_temp = pd.DataFrame()
    
    for i in range(8,38,6):
        started_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+1]) - pd.to_datetime(data.iloc[:, i])).dt.total_seconds() , True)
        docker_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+2]) - pd.to_datetime(data.iloc[:, i+1])).dt.total_seconds() , True)
        import_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+4]) - pd.to_datetime(data.iloc[:, i+3])).dt.total_seconds() , True)
        
    
    started_temp.columns = ['starttime{}'.format(i) for i in range(1, 6)]
    docker_temp.columns = ['dockertime{}'.format(i) for i in range(1, 6)]
    import_temp.columns = ['importtime{}'.format(i) for i in range(1, 6)]
    
    data = pd.concat([data, started_temp, docker_temp, import_temp], axis=1)
    
    return data

In [93]:
def extract_imports(imp):
    imp = imp.split("'")
    l = ""
    
    for i in imp:
        if i == "(" or i == ")" or i == ",)" or i == "," or i == "()" or i == ", ":
            continue
        else:
            if l == "":
                l = i
            else:
                l = l + ";" + i
    return l

In [94]:
def repair(data, install_libaries, approach):
    for i in range(0,207):
        if install_libaries[approach][i] == 0 :
            if data.installtime.iloc[i] > -1 :
                data.buldingtime.iloc[i] = data.buldingtime.iloc[i] - data.installtime.iloc[i]
                data.installtime.iloc[i] = -1

This is one experiment run with using the top 10 packages over all functions
The installationtime is set to minus -1 if there was no install as for native python packages as os

In [114]:
#approach = "all_top_10"
#approach = "vanilla_approach"
#approach = "cluster_top_10"
approach = "on_the_fly"
col = ['imports', 'buldingtime', 'installtime', 'starttime', 'dockertime',
       'importtime', 'starttime1', 'starttime2', 'starttime3', 'starttime4',
       'starttime5', 'dockertime1', 'dockertime2', 'dockertime3',
       'dockertime4', 'dockertime5', 'importtime1', 'importtime2',
       'importtime3', 'importtime4', 'importtime5']

In [115]:
install = pd.read_csv("experiments/installations.csv")
install_libaries = pd.DataFrame()
for index, row in install.iterrows():
    if pd.isna(row[1]):
        ins = set()
    else: 
        ins = set(row[1].split(";"))
    all = set(row[2].split(";"))
    if pd.isna(row[3]):
        clu = set()
    else: 
        clu = set(row[3].split(";"))
    install_libaries = pd.concat([install_libaries, pd.DataFrame([[row[0], len(ins), len(ins)-0, len(ins)-len(ins.intersection(all)), len(ins)-len(ins.intersection(clu))]])], ignore_index=True)
install_libaries.columns = ['imports', 'toInstall', 'vanilla_approach', 'all_top_10', 'cluster_top_10']

install_libaries

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/4029374880.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.isna(row[1]):
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/4029374880.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ins = set(row[1].split(";"))
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/4029374880.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  all

,imports,toInstall,vanilla_approach,all_top_10,cluster_top_10
0,argparse;copy;datetime;funcx;itertools;json;ma...,7,7,4,0
1,argparse;copy;funcx;itertools;json;matplotlib;...,6,6,4,0
2,argparse;sys,1,1,1,0
3,CLEAN,1,1,1,0
4,CLEAN;os,1,1,1,0
...,...,...,...,...,...
202,timeit,0,0,0,0
203,torch,1,1,0,0
204,torch;torchvision,2,2,1,2
205,traceback,0,0,0,0


In [116]:
data = pd.read_csv("new/"+approach+"/run10.csv")
data = prepare(data)
imp = data[col[0]]
data = data[col[1:]]

In [117]:
for i in range(9,0,-1):
    tmp = pd.read_csv("new/"+approach+"/run"+str(i)+".csv")
    tmp = prepare(tmp)
    tmp = tmp[col[1:]]
    data = (data + tmp)
data = data/10
data.insert(0, col[0],imp)

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/3790440972.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  docker_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+2]) - pd.to_datetime(data.iloc[:, i+1])).dt.total_seconds() , True)
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/3790440972.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  import_temp.insert(range(8,38,6).index(i), "", (pd.to_datetime(data.iloc[:, i+4]) - pd.to_datetime(data.iloc[:, i+3])).dt.total_seconds() , True)
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/3790440972.py:24: UserWarning: Could not infer format, so each element will be parsed individually, falli

In [118]:
data

,imports,buldingtime,installtime,starttime,dockertime,importtime,starttime1,starttime2,starttime3,starttime4,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
0,argparse;copy;datetime;funcx;itertools;json;ma...,0.9038,29.474880,0.286422,31.546118,0.597153,0.244161,0.253966,0.236762,0.233486,...,29.991276,30.281231,30.172795,30.461695,30.418261,0.582543,0.570461,0.571737,0.590476,0.579711
1,argparse;copy;funcx;itertools;json;matplotlib;...,0.8938,26.856138,0.281016,28.923401,0.600742,0.232982,0.254937,0.227090,0.242446,...,28.114228,27.694680,27.573183,27.635621,27.610565,0.554495,0.596445,0.551516,0.558455,0.572667
2,argparse;sys,0.8738,3.069121,0.312389,4.441564,0.012495,0.239555,0.241909,0.253181,0.254710,...,3.666234,3.655552,3.749648,3.639065,3.638947,0.013471,0.011920,0.012143,0.011776,0.011862
3,CLEAN,0.8738,3.183532,0.285327,4.476388,0.000479,0.245912,0.228590,0.238837,0.228608,...,3.661748,3.586962,3.710312,3.540172,3.598388,0.000275,0.000298,0.000321,0.000304,0.000354
4,CLEAN;os,0.8738,3.092641,0.289656,4.382008,0.000500,0.237560,0.247072,0.230491,0.233346,...,3.713493,3.499260,3.785342,3.662016,3.680913,0.000303,0.000284,0.000290,0.000304,0.000330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,timeit,0.8738,-1.000000,0.284803,1.077302,0.004087,0.252749,0.239477,0.247366,0.247674,...,1.041334,1.038434,1.039402,1.041190,1.038639,0.003304,0.002926,0.002936,0.003123,0.003117
203,torch,0.8738,2.249650,0.319151,138.822404,1.806577,0.248284,0.269873,0.261626,0.267771,...,146.356480,140.128239,141.189730,140.807511,135.811207,1.621907,1.689780,1.784300,1.685205,1.722938
204,torch;torchvision,0.8838,2.390106,0.303826,148.090439,2.856813,0.260541,0.256676,0.266048,0.264752,...,145.056128,152.767143,143.880632,147.327546,153.890664,2.719161,2.701361,2.743347,2.673605,2.629078
205,traceback,0.8738,-1.000000,0.298827,1.230919,0.151036,0.260113,0.252954,0.248917,0.251588,...,1.163996,1.141251,1.143180,1.177526,1.151118,0.122443,0.102626,0.103475,0.133405,0.110655


In [119]:
#repair(data, install_libaries, approach)

This is the mapping between imports and function
For this merge and the following ones, there is no need to think about on which join-operation. I cleaned the data beforehand. That is, there will be always a pair 
```python
pd.merge(func_mapping,data, on='imports')
```

In [120]:
func_mapping = pd.read_csv('experiments/function_mapping.csv')
func_mapping = func_mapping.drop(2)
func_mapping['func4.imports'] = func_mapping[func_mapping['func4.imports'].str.len() >= 0]['func4.imports'].map(lambda x: extract_imports(x))
func_mapping = func_mapping[func_mapping['func4.imports'] != ""]
func_mapping.rename(columns={'func4.imports': 'imports'}, inplace=True)
func_mapping.drop(columns=['Unnamed: 0'], inplace=True)

In [121]:
func_mapping.head()

,Unnamed: 0.1,func4.name,imports,Cluster
0,0,0-04-_check_slurm_job_statuses.py,subprocess,Job
3,3,873-04-call_mtrack.py,mtrack;sys,Data:_mtrack
4,4,43-04-make_corr_plots.py,json;os;xpcs_webplot,Sci:_Instruments
6,6,3-04-submit_slurm_job.py,os;subprocess,Job
7,7,397-01-rejection_sample.py,random,Trivial


Here is the mapping between functions and task
```python
pd.merge(func_mapping,task_mapping, on='func4.name')
```

In [122]:
task_mapping = pd.read_csv("funcx/task_function_mapping.csv")

In [123]:
task_mapping.head()

,task_uuid,func4.name,executiontime
0,000001d5-b3f0-4c17-8074-04f013f7bf52,0-04-_check_slurm_job_statuses.py,0.0243
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,0-04-_check_slurm_job_statuses.py,0.0331
2,00002642-f850-42b4-9e1e-2698c3396498,0-04-_check_slurm_job_statuses.py,0.0432
3,00002deb-cf79-43ed-8aff-3281938809fe,0-04-_check_slurm_job_statuses.py,0.0268
4,0000754b-898e-43dc-9b55-35d1a6fe6633,0-04-_check_slurm_job_statuses.py,0.0284


Here we have the timestamp for each task
For merging
```python
pd.merge(task_mapping,task_mapping, on='task_uuid')
```

In [124]:
task = pd.read_csv("funcx/tasks_pt1.csv")
task = pd.concat([task, pd.read_csv("funcx/tasks_pt2.csv")], ignore_index=True)
task['received'] = task['received'].interpolate()

In [125]:
task.head()

,task_uuid,user_id,endpoint_uuid,received
0,000001d5-b3f0-4c17-8074-04f013f7bf52,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.681417e+18
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679386e+18
2,00002642-f850-42b4-9e1e-2698c3396498,286.0,cbc1f8d4-71b2-4a41-8ed4-9c75421eb89f,1.683246e+18
3,00002deb-cf79-43ed-8aff-3281938809fe,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.677897e+18
4,0000754b-898e-43dc-9b55-35d1a6fe6633,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679277e+18


Here are information about the endpoints
For merging:
```python
pd.merge(task,endpoints, on='endpoint_uuid')
```

In [126]:
endpoints = pd.read_csv("funcx/endpoints.csv")

In [127]:
endpoints.head()

,endpoint_uuid,X_type,ip,cluster
0,00000000-0000-4000-8000-000011110000,LocalProvider,216.80.70.252,6
1,001fecc2-1393-4bd9-950e-5645e15e3a53,SlurmProvider,141.142.140.194,10
2,006f558d-df82-45c2-b2d5-94274ef41a69,SlurmProvider,141.142.140.194,10
3,00824163-569c-4f0d-953d-097cd3b7179d,NaN,159.226.43.47,10
4,01a3324e-7ad9-4563-b598-0c2499068a39,LocalProvider,128.135.204.6,8


Example merging in the end:
```python
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
```

In [128]:
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')

In [129]:
sorted_df = r.sort_values(by=["received"], ascending=True)
sorted_df = sorted_df.dropna()

In [130]:
grouped_indices = sorted_df.groupby('func4.name').groups
# Iterate over each function name and remove the first index
for func_name, indices in grouped_indices.items():
    sorted_df.buldingtime[indices[1:]] = 0

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/1479014251.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sorted_df.buldingtime[indices[1:]] = 0
/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/147901425

In [133]:
if approach == "on_the_fly":
    sorted_df.buldingtime[sorted_df.installtime>0] += sorted_df.installtime[sorted_df.installtime>0]

/var/folders/86/fyd29lqj0033vg6drb2b412m0000gn/T/ipykernel_57996/694351189.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sorted_df.buldingtime[sorted_df.installtime>0] += sorted_df.installtime[sorted_df.installtime>0]
/var/folders/86/f

In [134]:
sorted_df

,task_uuid,user_id,endpoint_uuid,received,func4.name,executiontime,Unnamed: 0.1,imports,Cluster,buldingtime,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
1109608,2dda391f-cb85-4e2f-8ae7-c9c603a821dd,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669597e+18,296-04-tar.py,0.0021,436,os;pathlib;tarfile,Sci:_Instruments,3.992972,...,3.724665,3.672935,3.636417,3.619117,3.653855,0.021270,0.020972,0.02133,0.021870,0.020667
1109630,ac3f3e5e-aeff-4236-a3a5-223dcf00aba4,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669598e+18,296-04-tar.py,0.0017,436,os;pathlib;tarfile,Sci:_Instruments,3.119172,...,3.724665,3.672935,3.636417,3.619117,3.653855,0.021270,0.020972,0.02133,0.021870,0.020667
1109604,1347620d-56a1-4d30-b8d3-32ded9e08b47,458.0,396e6fd2-ed3d-4e84-b7a6-b8bc7d98d230,1.669598e+18,296-04-tar.py,0.0020,436,os;pathlib;tarfile,Sci:_Instruments,3.119172,...,3.724665,3.672935,3.636417,3.619117,3.653855,0.021270,0.020972,0.02133,0.021870,0.020667
1109605,1fbeacd4-b3e9-4bb9-bc1c-92fc9abd238f,600.0,674446c2-4dc6-41a6-b767-bf6ce67df702,1.669598e+18,296-04-tar.py,0.0095,436,os;pathlib;tarfile,Sci:_Instruments,3.119172,...,3.724665,3.672935,3.636417,3.619117,3.653855,0.021270,0.020972,0.02133,0.021870,0.020667
1109606,285a505d-66ae-4e5c-88a3-e70794e425ae,600.0,674446c2-4dc6-41a6-b767-bf6ce67df702,1.669598e+18,296-04-tar.py,0.0081,436,os;pathlib;tarfile,Sci:_Instruments,3.119172,...,3.724665,3.672935,3.636417,3.619117,3.653855,0.021270,0.020972,0.02133,0.021870,0.020667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615572,e02fd7ab-c245-4120-9ffa-7fb449881a15,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.688342e+18,0-04-_check_slurm_job_statuses.py,0.0297,0,subprocess,Job,0.000000,...,1.167932,1.187134,1.191699,1.175575,1.172227,0.127317,0.147771,0.14759,0.131461,0.132080
321864,752a5e0f-3468-49e4-8dbc-36c148f38eba,286.0,cbc1f8d4-71b2-4a41-8ed4-9c75421eb89f,1.688342e+18,0-04-_check_slurm_job_statuses.py,0.0204,0,subprocess,Job,0.000000,...,1.167932,1.187134,1.191699,1.175575,1.172227,0.127317,0.147771,0.14759,0.131461,0.132080
283169,670c710d-6e2a-434d-8db9-500d00f8b92b,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.688342e+18,0-04-_check_slurm_job_statuses.py,0.0289,0,subprocess,Job,0.000000,...,1.167932,1.187134,1.191699,1.175575,1.172227,0.127317,0.147771,0.14759,0.131461,0.132080
363374,844cfb85-6213-443d-8db1-261d20cbac9f,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.688342e+18,0-04-_check_slurm_job_statuses.py,0.0230,0,subprocess,Job,0.000000,...,1.167932,1.187134,1.191699,1.175575,1.172227,0.127317,0.147771,0.14759,0.131461,0.132080


In [135]:
sorted_df.to_csv(approach+"_data_sorted.csv")